In [104]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [105]:
os.chdir('D:\\MDA\\Course\\Boot Camp\\SCMA 632\\Assignments\\A1b')


In [106]:
ipl = pd.read_csv('IPL_ball_by_ball_updated till 2024.csv', low_memory = False)

In [107]:
ipl_salary = pd.read_excel('IPL SALARIES 2024.xlsx')

In [108]:
ipl_salary.head(2)

,Player,Salary,Rs,international,iconic
0,Abhishek Porel,20 lakh,20,0,NaN
1,Anrich Nortje,6.5 crore,650,1,NaN


In [109]:
ipl.head(2)

,Match id,Date,Season,Batting team,Bowling team,Innings No,Ball No,Bowler,Striker,Non Striker,runs_scored,extras,type of extras,score,score/wicket,wicket_confirmation,wicket_type,fielders_involved,Player Out
0,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.1,P Kumar,SC Ganguly,BB McCullum,0,1,legbyes,1,1/0,0,NaN,NaN,NaN
1,335982,18-04-2008,2007/08,Kolkata Knight Riders,Royal Challengers Bangalore,1,0.2,P Kumar,BB McCullum,SC Ganguly,0,0,NaN,1,1/0,0,NaN,NaN,NaN


In [110]:
grp_data = ipl.groupby(['Season','Innings No', 'Striker', 
                        'Bowler']).agg({'runs_scored': sum,
                                        'wicket_confirmation':sum}).reset_index()

In [111]:
grp_data.head()

,Season,Innings No,Striker,Bowler,runs_scored,wicket_confirmation
0,2007/08,1,A Chopra,DP Vijaykumar,1,0
1,2007/08,1,A Chopra,DW Steyn,1,1
2,2007/08,1,A Chopra,GD McGrath,2,0
3,2007/08,1,A Chopra,PJ Sangwan,6,1
4,2007/08,1,A Chopra,RP Singh,9,0


In [112]:
player_runs = grp_data.groupby(['Season', 'Striker'])['runs_scored'].sum().reset_index()
player_wickets = grp_data.groupby(['Season', 'Bowler'])['wicket_confirmation'].sum().reset_index()

In [113]:
player_runs[player_runs['Season'] == '2023'].sort_values(by = 'runs_scored', ascending = False)

,Season,Striker,runs_scored
2423,2023,Shubman Gill,890
2313,2023,F du Plessis,730
2311,2023,DP Conway,672
2433,2023,V Kohli,639
2443,2023,YBK Jaiswal,625
...,...,...,...
2404,2023,RP Meredith,0
2372,2023,Mohsin Khan,0
2307,2023,DG Nalkande,0
2429,2023,TU Deshpande,0


In [114]:
player_wickets[player_wickets['Season'] == '2023'].sort_values(by = 'wicket_confirmation', ascending = False)

,Season,Bowler,wicket_confirmation
1750,2023,MM Sharma,31
1755,2023,Mohammed Shami,28
1782,2023,Rashid Khan,28
1797,2023,TU Deshpande,24
1770,2023,PP Chawla,23
...,...,...,...
1776,2023,R Tewatia,0
1709,2023,H Sharma,0
1708,2023,Gurnoor Brar,0
1702,2023,DJ Hooda,0


In [115]:
top_run_getters = player_runs.groupby('Season').apply(lambda x: x.nlargest(3, 
                'runs_scored')).reset_index(drop=True)
bottom_wicket_takers = player_wickets.groupby('Season').apply(lambda x: x.nlargest(3,
                'wicket_confirmation')).reset_index(drop=True)
print("Top Three Run Getters:")
print(top_run_getters)
print("Top Three Wicket Takers:")
print(bottom_wicket_takers)

Top Three Run Getters:
     Season          Striker  runs_scored
0   2007/08         SE Marsh          616
1   2007/08        G Gambhir          534
2   2007/08    ST Jayasuriya          514
3      2009        ML Hayden          572
4      2009     AC Gilchrist          495
5      2009   AB de Villiers          465
6   2009/10     SR Tendulkar          618
7   2009/10        JH Kallis          572
8   2009/10         SK Raina          528
9      2011         CH Gayle          608
10     2011          V Kohli          557
11     2011     SR Tendulkar          553
12     2012         CH Gayle          733
13     2012        G Gambhir          590
14     2012         S Dhawan          569
15     2013       MEK Hussey          733
16     2013         CH Gayle          720
17     2013          V Kohli          639
18     2014       RV Uthappa          660
19     2014         DR Smith          566
20     2014       GJ Maxwell          552
21     2015        DA Warner          562
22     2015

In [116]:
ipl_year_id = pd.DataFrame(columns=["id", "year"])
ipl_year_id["id"] = ipl["Match id"]
ipl_year_id["year"] = pd.to_datetime(ipl["Date"], dayfirst=True).dt.year

In [117]:
#create a copy of ipl_bbbc dataframe
ipl_bbbc= ipl.copy()

In [118]:
ipl_bbbc['year'] = pd.to_datetime(ipl["Date"], dayfirst=True).dt.year


In [119]:
ipl_bbbc[["Match id", "year", "runs_scored","wicket_confirmation"
          ,"Bowler",'Striker']].head()


,Match id,year,runs_scored,wicket_confirmation,Bowler,Striker
0,335982,2008,0,0,P Kumar,SC Ganguly
1,335982,2008,0,0,P Kumar,BB McCullum
2,335982,2008,0,0,P Kumar,BB McCullum
3,335982,2008,0,0,P Kumar,BB McCullum
4,335982,2008,0,0,P Kumar,BB McCullum


In [120]:
import scipy.stats as st

def get_best_distribution(data):
    dist_names = ['alpha','beta','betaprime','burr12','crystalball',
                  'dgamma','dweibull','erlang','exponnorm','f','fatiguelife',
                  'gamma','gengamma','gumbel_l','johnsonsb','kappa4',
                  'lognorm','nct','norm','norminvgauss','powernorm','rice',
                  'recipinvgauss','t','trapz','truncnorm']
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        # Applying the Kolmogorov-Smirnov test
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))
    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value
    print("\nBest fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))
    return best_dist, best_p, params[best_dist]

In [121]:
total_run_each_year = ipl_bbbc.groupby(["year", "Striker"])["runs_scored"].sum().reset_index()


In [122]:
total_run_each_year

,year,Striker,runs_scored
0,2008,A Chopra,42
1,2008,A Kumble,13
2,2008,A Mishra,37
3,2008,A Mukund,0
4,2008,A Nehra,3
...,...,...,...
2593,2024,Vijaykumar Vyshak,1
2594,2024,WG Jacks,176
2595,2024,WP Saha,135
2596,2024,Washington Sundar,0


In [123]:
total_run_each_year.sort_values(["year", "runs_scored"], ascending=False, inplace=True)
print(total_run_each_year)

      year          Striker  runs_scored
2549  2024       RD Gaikwad          509
2589  2024          V Kohli          500
2470  2024  B Sai Sudharsan          418
2502  2024         KL Rahul          406
2555  2024          RR Pant          398
...    ...              ...          ...
58    2008         L Balaji            0
66    2008   M Muralitharan            0
75    2008         MM Patel            0
107   2008      S Sreesanth            0
136   2008           U Kaul            0

[2598 rows x 3 columns]


In [124]:
total_run_each_year.head(11)

,year,Striker,runs_scored
2549,2024,RD Gaikwad,509
2589,2024,V Kohli,500
2470,2024,B Sai Sudharsan,418
2502,2024,KL Rahul,406
2555,2024,RR Pant,398
2535,2024,PD Salt,392
2572,2024,SV Samson,385
2569,2024,SP Narine,372
2562,2024,S Dube,350
2587,2024,Tilak Varma,343


In [125]:
list_top_batsman_last_three_year = {}
for i in total_run_each_year["year"].unique()[:3]:list_top_batsman_last_three_year[i] = total_run_each_year[total_run_each_year.year == i][:3]["Striker"].unique().tolist()


In [126]:
list_top_batsman_last_three_year


{2024: ['RD Gaikwad', 'V Kohli', 'B Sai Sudharsan'],
 2023: ['Shubman Gill', 'F du Plessis', 'DP Conway'],
 2022: ['JC Buttler', 'KL Rahul', 'Q de Kock']}

In [127]:
import warnings
warnings.filterwarnings('ignore')
runs = ipl_bbbc.groupby(['Striker','Match id'])[['runs_scored']].sum().reset_index()


for key in list_top_batsman_last_three_year:
    for Striker in list_top_batsman_last_three_year[key]:
        print("************************")
        print("year:", key, " Batsman:", Striker)
        get_best_distribution(runs[runs["Striker"] == Striker]["runs_scored"])
        print("\n\n")

************************
year: 2024  Batsman: RD Gaikwad
p value for alpha = 2.599259711013304e-20
p value for beta = 0.02041902689492403
p value for betaprime = 0.019503763598668566
p value for burr12 = 0.46882020698395865
p value for crystalball = 0.24953646987270484
p value for dgamma = 0.1570743843120962
p value for dweibull = 0.20046582403736823
p value for erlang = 1.893799588395604e-06
p value for exponnorm = 0.4644304230917985
p value for f = 1.3560920695663998e-07
p value for fatiguelife = 1.304427037367869e-14
p value for gamma = 0.005830868576003678
p value for gengamma = 0.015331622187826577
p value for gumbel_l = 0.05546236480086586
p value for johnsonsb = 4.646964117947127e-13
p value for kappa4 = 0.006363220770325362
p value for lognorm = 1.1719355665219537e-16
p value for nct = 0.5881570496217807
p value for norm = 0.24953651809309751
p value for norminvgauss = 0.5538573365184996
p value for powernorm = 0.1788753268739086
p value for rice = 0.18287532184336575
p value f

In [128]:
total_wicket_each_year = ipl_bbbc.groupby(["year", "Bowler"])["wicket_confirmation"].sum().reset_index()


In [129]:
total_wicket_each_year.sort_values(["year", "wicket_confirmation"], ascending=False, inplace=True)
print(total_wicket_each_year)

      year             Bowler  wicket_confirmation
1836  2024           HV Patel                   19
1875  2024       Mukesh Kumar                   15
1822  2024     Arshdeep Singh                   14
1842  2024          JJ Bumrah                   14
1876  2024  Mustafizur Rahman                   14
...    ...                ...                  ...
16    2008           CL White                    0
41    2008             K Goel                    0
43    2008          LPC Silva                    0
60    2008       Pankaj Singh                    0
90    2008        VS Yeligati                    0

[1929 rows x 3 columns]


In [130]:
list_top_bowler_last_three_year = {}
for i in total_wicket_each_year["year"].unique()[:3]:list_top_bowler_last_three_year[i] = total_wicket_each_year [total_wicket_each_year.year == i][:3]["Bowler"].unique().tolist()
list_top_bowler_last_three_year

{2024: ['HV Patel', 'Mukesh Kumar', 'Arshdeep Singh'],
 2023: ['MM Sharma', 'Mohammed Shami', 'Rashid Khan'],
 2022: ['YS Chahal', 'PWH de Silva', 'K Rabada']}

In [131]:
import warnings
warnings.filterwarnings('ignore')
wickets = ipl_bbbc.groupby(['Bowler','Match id'])[['wicket_confirmation']].sum().reset_index()

for key in list_top_bowler_last_three_year:
    for bowler in list_top_bowler_last_three_year[key]:
        print("************************")
        print("year:", key, " Bowler:", bowler)
        get_best_distribution(wickets[wickets["Bowler"] == bowler]["wicket_confirmation"])
        print("\n\n")


************************
year: 2024  Bowler: HV Patel
p value for alpha = 0.0002993252328930706
p value for beta = 2.777571908776589e-19
p value for betaprime = 1.7052883875145053e-30
p value for burr12 = 5.427998338605459e-15
p value for crystalball = 1.1109118198587684e-05
p value for dgamma = 4.375428528574276e-05
p value for dweibull = 1.8553295107771936e-05
p value for erlang = 5.473635282991912e-24
p value for exponnorm = 0.0002813279943461815
p value for f = 1.9012983291282487e-09
p value for fatiguelife = 1.9734428958773156e-05
p value for gamma = 1.470787431589663e-16
p value for gengamma = 1.4345058849022962e-16
p value for gumbel_l = 4.541523588271283e-05
p value for johnsonsb = 2.827201329331457e-51
p value for kappa4 = 9.177530010006471e-23
p value for lognorm = 5.2162358572043325e-22
p value for nct = 0.0001960277304576293
p value for norm = 1.1109124960635979e-05
p value for norminvgauss = 3.811196478020768e-05
p value for powernorm = 3.2186417463058256e-05
p value for r

In [132]:
import warnings
warnings.filterwarnings('ignore')

wickets = ipl_bbbc.groupby(['Bowler','Match id'])[['wicket_confirmation']].sum().reset_index()

# Choose the bowler you want to analyze (replace with desired bowler name)
chosen_bowler = "Arshdeep Singh"  # Replace with your chosen bowler's name

print("************************")
print("Best fit distribution for wickets taken by:", chosen_bowler)
get_best_distribution(wickets[wickets["Bowler"] == chosen_bowler]["wicket_confirmation"])
print("\n\n")

************************
Best fit distribution for wickets taken by: Arshdeep Singh
p value for alpha = 0.002547644307209551
p value for beta = 3.7725133611153275e-15
p value for betaprime = 5.062381659741898e-22
p value for burr12 = 4.603956720503075e-14
p value for crystalball = 0.0002501762149918564
p value for dgamma = 0.00028566200697101806
p value for dweibull = 0.0016211491850549598
p value for erlang = 2.269289539862191e-12
p value for exponnorm = 0.0019097947631203649
p value for f = 0.000227258408802241
p value for fatiguelife = 2.169103029961132e-15
p value for gamma = 6.618486511618167e-29
p value for gengamma = 5.948936850168967e-23
p value for gumbel_l = 0.00026864389982599567
p value for johnsonsb = 5.472387372640376e-24
p value for kappa4 = 8.181970339328129e-12
p value for lognorm = 1.9909678840157557e-12
p value for nct = 0.0014257070102449143
p value for norm = 0.00025017539197677184
p value for norminvgauss = 0.0001290021448063343
p value for powernorm = 0.000471377

In [133]:
total_run_each_year

,year,Striker,runs_scored
2549,2024,RD Gaikwad,509
2589,2024,V Kohli,500
2470,2024,B Sai Sudharsan,418
2502,2024,KL Rahul,406
2555,2024,RR Pant,398
...,...,...,...
58,2008,L Balaji,0
66,2008,M Muralitharan,0
75,2008,MM Patel,0
107,2008,S Sreesanth,0


In [134]:
total_wicket_each_year

,year,Bowler,wicket_confirmation
1836,2024,HV Patel,19
1875,2024,Mukesh Kumar,15
1822,2024,Arshdeep Singh,14
1842,2024,JJ Bumrah,14
1876,2024,Mustafizur Rahman,14
...,...,...,...
16,2008,CL White,0
41,2008,K Goel,0
43,2008,LPC Silva,0
60,2008,Pankaj Singh,0


In [135]:
R2024 =total_run_each_year[total_run_each_year['year']==2024]


In [136]:
W2024 = total_wicket_each_year[total_wicket_each_year['year']==2024]

In [137]:
R2024

,year,Striker,runs_scored
2549,2024,RD Gaikwad,509
2589,2024,V Kohli,500
2470,2024,B Sai Sudharsan,418
2502,2024,KL Rahul,406
2555,2024,RR Pant,398
...,...,...,...
2523,2024,Mayank Dagar,0
2527,2024,Mukesh Kumar,0
2560,2024,Ravi Bishnoi,0
2583,2024,TA Boult,0


In [138]:
W2024

,year,Bowler,wicket_confirmation
1836,2024,HV Patel,19
1875,2024,Mukesh Kumar,15
1822,2024,Arshdeep Singh,14
1842,2024,JJ Bumrah,14
1876,2024,Mustafizur Rahman,14
...,...,...,...
1911,2024,Sumit Kumar,0
1912,2024,Suyash Sharma,0
1918,2024,Tilak Varma,0
1920,2024,Umran Malik,0


In [139]:
pip install fuzzywuzzy


Note: you may need to restart the kernel to use updated packages.


In [140]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = ipl_salary.copy()
df_runs = R2024.copy()
df_wickets = W2024.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Striker'].tolist()))


# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Striker')


In [141]:
# Convert to DataFrame
df_salary = ipl_salary.copy()
df_runs = R2024.copy()
df_wickets = W2024.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_wickets['Bowler'].tolist()))


# Merge the DataFrames on the matched names
df_merged1 = pd.merge(df_salary, df_wickets, left_on='Matched_Player', right_on='Bowler')

In [152]:
df_merged1.columns

Index(['Player', 'Salary', 'Rs', 'international', 'iconic', 'Matched_Player',
       'year', 'Bowler', 'wicket_confirmation'],
      dtype='object')

In [153]:
df_merged.columns

Index(['Player', 'Salary', 'Rs', 'international', 'iconic', 'Matched_Player',
       'year', 'Striker', 'runs_scored'],
      dtype='object')

In [144]:
df_merged1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               81 non-null     object 
 1   Salary               81 non-null     object 
 2   Rs                   81 non-null     int64  
 3   international        81 non-null     int64  
 4   iconic               0 non-null      float64
 5   Matched_Player       81 non-null     object 
 6   year                 81 non-null     int32  
 7   Bowler               81 non-null     object 
 8   wicket_confirmation  81 non-null     int64  
dtypes: float64(1), int32(1), int64(3), object(4)
memory usage: 5.5+ KB


In [145]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          111 non-null    object 
 1   Salary          111 non-null    object 
 2   Rs              111 non-null    int64  
 3   international   111 non-null    int64  
 4   iconic          0 non-null      float64
 5   Matched_Player  111 non-null    object 
 6   year            111 non-null    int32  
 7   Striker         111 non-null    object 
 8   runs_scored     111 non-null    int64  
dtypes: float64(1), int32(1), int64(3), object(4)
memory usage: 7.5+ KB


In [146]:
# Calculate the correlation
correlation = df_merged['Rs'].corr(df_merged['runs_scored'])

print("Correlation between Salary and Runs:", correlation)

Correlation between Salary and Runs: 0.3061248376582168


### Significant Difference Between the Salaries of the Top 10 Batsmen and Top Wicket-Taking Bowlers Over the Last Three Years

In [159]:
df_merged

,Player,Salary,Rs,international,iconic,Matched_Player,year,Striker,runs_scored
0,Abhishek Porel,20 lakh,20,0,NaN,Abishek Porel,2024,Abishek Porel,202
1,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2024,A Nortje,4
2,Axar Patel,9 crore,900,0,NaN,AR Patel,2024,AR Patel,149
3,David Warner,6.25 crore,625,1,NaN,TH David,2024,TH David,217
4,David Miller,3 crore,300,1,NaN,TH David,2024,TH David,217
...,...,...,...,...,...,...,...,...,...
106,Anmolpreet Singh,20 lakh,20,0,NaN,Anmolpreet Singh,2024,Anmolpreet Singh,0
107,Heinrich Klaasen,5.25 crore,525,1,NaN,H Klaasen,2024,H Klaasen,295
108,Marco Jansen,4.2 crore,420,1,NaN,M Jansen,2024,M Jansen,1
109,Rahul Tripathi,8.5 crore,850,0,NaN,RA Tripathi,2024,RA Tripathi,31


In [158]:
df_merged1

,Player,Salary,Rs,international,iconic,Matched_Player,year,Bowler,wicket_confirmation
0,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2024,A Nortje,7
1,Axar Patel,9 crore,900,0,NaN,AR Patel,2024,AR Patel,9
2,Ishant Sharma,50 lakh,50,0,NaN,I Sharma,2024,I Sharma,8
3,Mohit Sharma,50 lakhs,50,0,NaN,I Sharma,2024,I Sharma,8
4,Rohit Sharma,16 Crore,1600,0,NaN,I Sharma,2024,I Sharma,8
...,...,...,...,...,...,...,...,...,...
76,Marco Jansen,4.2 crore,420,1,NaN,M Jansen,2024,M Jansen,1
77,Mayank Markande,50 lakh,50,0,NaN,M Markande,2024,M Markande,8
78,Nitish Kumar Reddy,20 lakh,20,0,NaN,Nithish Kumar Reddy,2024,Nithish Kumar Reddy,3
79,Umran Malik,4 crore,400,0,NaN,Umran Malik,2024,Umran Malik,0


In [165]:
merged_df

,Player_x,Salary_x,Rs_x,international_x,iconic_x,Matched_Player,year_x,Striker,runs_scored,Player_y,Salary_y,Rs_y,international_y,iconic_y,year_y,Bowler,wicket_confirmation
0,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2024,A Nortje,4,Anrich Nortje,6.5 crore,650,1,NaN,2024,A Nortje,7
1,Axar Patel,9 crore,900,0,NaN,AR Patel,2024,AR Patel,149,Axar Patel,9 crore,900,0,NaN,2024,AR Patel,9
2,Ishant Sharma,50 lakh,50,0,NaN,I Sharma,2024,I Sharma,1,Ishant Sharma,50 lakh,50,0,NaN,2024,I Sharma,8
3,Ishant Sharma,50 lakh,50,0,NaN,I Sharma,2024,I Sharma,1,Mohit Sharma,50 lakhs,50,0,NaN,2024,I Sharma,8
4,Ishant Sharma,50 lakh,50,0,NaN,I Sharma,2024,I Sharma,1,Rohit Sharma,16 Crore,1600,0,NaN,2024,I Sharma,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Bhuvneshwar Kumar,4.2 crore,420,0,NaN,B Kumar,2024,B Kumar,23,Rajan Kumar,70 lakh,70,0,NaN,2024,B Kumar,5
68,Bhuvneshwar Kumar,4.2 crore,420,0,NaN,B Kumar,2024,B Kumar,23,Bhuvneshwar Kumar,4.2 crore,420,0,NaN,2024,B Kumar,5
69,Abhishek Sharma,6.5 crore,650,0,NaN,Abhishek Sharma,2024,Abhishek Sharma,303,Abhishek Sharma,6.5 crore,650,0,NaN,2024,Abhishek Sharma,0
70,Marco Jansen,4.2 crore,420,1,NaN,M Jansen,2024,M Jansen,1,Marco Jansen,4.2 crore,420,1,NaN,2024,M Jansen,1


In [164]:
merged_df.columns.tolist()

['Player_x',
 'Salary_x',
 'Rs_x',
 'international_x',
 'iconic_x',
 'Matched_Player',
 'year_x',
 'Striker',
 'runs_scored',
 'Player_y',
 'Salary_y',
 'Rs_y',
 'international_y',
 'iconic_y',
 'year_y',
 'Bowler',
 'wicket_confirmation']

In [166]:
from fuzzywuzzy import process  # Assuming fuzzywuzzy is already imported

# Filter last three years' data (assuming 'year' column exists)
last_three_years_data = merged_df[merged_df['year_x'].isin([2022, 2023, 2024])]

# Function to get top players
def get_top_players(data, n, role):
  if role == "Matched_Player":
    top_players = last_three_years_data.groupby('Matched_Player')['runs_scored'].sum().sort_values(ascending=False).head(n)
  elif role == "Bowler":
    top_players = last_three_years_data.groupby('Bowler')['wicket_confirmation'].sum().sort_values(ascending=False).head(n)
  else:
    raise ValueError("Invalid role. Choose 'Matched_Player' or 'Bowler'")
  return top_players.index.tolist()

# Get top 10 batsmen and bowlers
top_10_batsmen = get_top_players(last_three_years_data, 10, "Matched_Player")
top_10_bowlers = get_top_players(last_three_years_data, 10, "Bowler")

# Calculate average salary (assuming 'Rs_x' column holds salary information)
avg_salary_batsmen = last_three_years_data[last_three_years_data['Matched_Player'].isin(top_10_batsmen)]['Rs_x'].mean()
avg_salary_bowlers = last_three_years_data[last_three_years_data['Bowler'].isin(top_10_bowlers)]['Rs_x'].mean()

# Calculate difference
difference = avg_salary_batsmen - avg_salary_bowlers

# Print result
print("Significant Difference in Salary (Batsmen - Bowlers):", difference)


Significant Difference in Salary (Batsmen - Bowlers): 221.42676767676772
